Added laser tables to `0_raw_laser` and `1_filtered_data` manually + excel formatting

In [89]:
import pandas as pd
import numpy as np
from ODR import odr_fit
from scipy.stats import chi2

math latex in markdown test:
$$
\left(\frac{a}{b}\right)
$$

### Stick measurements
Get pixel length in mm

In [90]:
df = pd.read_excel("data.xlsx", sheet_name="2_stick_measurements", index_col=0)

stick_diameter_in_mm = 1.25
stick_diameter_measurements = df["diameter_length-pixels"].values

# Calculate stick diameter in pixels, using mean
stick_diameter_in_pixels = np.mean(stick_diameter_measurements)

# Calculate the error:
# Calculating the standard error of the mean, applying Bessel's correction
std_dev = np.std(stick_diameter_measurements, ddof=1)
sem = std_dev / np.sqrt(len(stick_diameter_measurements))
# Getting the measurement error from the data - it is constant for all measurements
measurement_error_in_pixels = df["diameter_length_error-pixels"].values[0]
# Calculating the total error
stick_diameter_error_in_pixels = np.sqrt(measurement_error_in_pixels ** 2 + sem ** 2)

# Calculate the pixel to mm conversion factor
pixel_to_mm = stick_diameter_in_mm / stick_diameter_in_pixels
pixel_to_mm_error = pixel_to_mm * stick_diameter_error_in_pixels / stick_diameter_in_pixels

pixel_to_mm, pixel_to_mm_error

(np.float64(0.0005934615365156852), np.float64(1.6843048484052963e-06))

### Particle mass measurement
Measure the particle's diameter

In [91]:
df = pd.read_excel("data.xlsx", sheet_name="3_particle_measurements", index_col=0)
particle_diameter_measurements = df["diameter-pixels"].values

# Calculate particle diameter in pixels
particle_diameter_in_pixels = np.mean(particle_diameter_measurements)

# Calculate the statistical error
particle_diameter_error_in_pixels = np.std(particle_diameter_measurements, ddof=1) / np.sqrt(len(particle_diameter_measurements))

# Particle diameter in mm
particle_diameter_in_mm = particle_diameter_in_pixels * pixel_to_mm
particle_diameter_error_in_mm = np.sqrt(
    (particle_diameter_error_in_pixels * pixel_to_mm) ** 2 + (particle_diameter_in_pixels * pixel_to_mm_error) ** 2
)

particle_diameter_in_mm, particle_diameter_error_in_mm

(np.float64(0.029475256313612364), np.float64(0.0007181409158178655))

Calculate the particle's volume

In [92]:
# Assuming the particle is a sphere:
particle_volume_in_mm_cubed = (np.pi * (particle_diameter_in_mm ** 3)) / 6
particle_volume_error_in_mm_cubed = particle_volume_in_mm_cubed * 3 * (particle_diameter_error_in_mm / particle_diameter_in_mm)

particle_volume_in_mm_cubed, particle_volume_error_in_mm_cubed

(np.float64(1.3408228218918054e-05), np.float64(9.800420926126294e-07))

Calculate the particle's mass using known density

In [93]:
PARTICLE_DENSITY_IN_KG_OVER_M_CUBED = 510
PARTICLE_DENSITY_ERROR_IN_KG_OVER_M_CUBED = 40

particle_volume_in_m_cubed = particle_volume_in_mm_cubed * 1e-9
particle_volume_error_in_m_cubed = particle_volume_error_in_mm_cubed * 1e-9

particle_mass_in_kg = particle_volume_in_m_cubed * PARTICLE_DENSITY_IN_KG_OVER_M_CUBED
particle_mass_error_in_kg = np.sqrt(
    (particle_volume_error_in_m_cubed * PARTICLE_DENSITY_IN_KG_OVER_M_CUBED) ** 2 + (particle_volume_in_m_cubed * PARTICLE_DENSITY_ERROR_IN_KG_OVER_M_CUBED) ** 2
)

particle_mass_in_kg, particle_mass_error_in_kg

(np.float64(6.838196391648207e-12), np.float64(7.331237504400842e-13))

### Particle charge
Calculate the suspension DC using the monitor DC measurements when the particle was suspended. Ignore AC effect.

In [94]:
df = pd.read_excel("data.xlsx", sheet_name="4_charge_measurements", index_col=0)
mon_dc_measurements = df["dc-kilovolts"].values

# Calculate the DC needed to suspend the particle
mon_dc_in_volts = np.mean(mon_dc_measurements * 1e+3)

# Calculate the error
mon_dc_error_in_volts = np.std(mon_dc_measurements * 1e+3, ddof=1) / np.sqrt(len(mon_dc_measurements))

def convert_mon_dc_to_suspension_dc(mon_dc):
    return 82 * (mon_dc - 0.06)

suspension_dc_in_volts = convert_mon_dc_to_suspension_dc(mon_dc_in_volts)
suspension_dc_error_in_volts = suspension_dc_in_volts * mon_dc_error_in_volts / mon_dc_in_volts

mon_dc_in_volts, mon_dc_error_in_volts, suspension_dc_in_volts, suspension_dc_error_in_volts

(np.float64(676.6633333333333),
 np.float64(1.1900186739897933),
 np.float64(55481.473333333335),
 np.float64(97.57287867496896))

Calculate the electric field used to hold the particle

In [95]:
def convert_mon_dc_to_suspension_field(mon_dc):
    return 4850 * (mon_dc - 0.06)

suspension_field_in_volts_per_m = convert_mon_dc_to_suspension_field(mon_dc_in_volts)
suspension_field_error_in_volts_per_m = suspension_field_in_volts_per_m * mon_dc_error_in_volts / mon_dc_in_volts 

suspension_field_in_volts_per_m, suspension_field_error_in_volts_per_m

(np.float64(3281526.166666667), np.float64(5771.078799678043))

Calculate the charge over mass and then the charge itself

In [96]:
GRAVITATIONAL_ACCELERATION_IN_M_PER_SEC_SQUARED = 9.81

particle_charge_per_mass_in_c_over_kg = GRAVITATIONAL_ACCELERATION_IN_M_PER_SEC_SQUARED / suspension_field_in_volts_per_m
particle_charge_per_mass_error_in_c_over_kg = (
    particle_charge_per_mass_in_c_over_kg * (suspension_field_error_in_volts_per_m / suspension_field_in_volts_per_m)
)

particle_charge_per_mass_in_c_over_kg, particle_charge_per_mass_error_in_c_over_kg

(np.float64(2.9894626773507872e-06), np.float64(5.257439314345826e-09))

In [97]:
particle_charge_in_coulomb = particle_charge_per_mass_in_c_over_kg * particle_mass_in_kg
particle_charge_error_in_coulomb = np.sqrt(
    (particle_charge_per_mass_error_in_c_over_kg * particle_mass_in_kg) ** 2 + (particle_charge_per_mass_in_c_over_kg * particle_mass_error_in_kg) ** 2
)

particle_charge_in_coulomb, particle_charge_error_in_coulomb

(np.float64(2.044253289322714e-17), np.float64(2.1919409404381304e-18))

### Length and height measurements

In [98]:
df = pd.read_excel("data.xlsx", sheet_name="1_filtered_data", index_col=0)
# Dropping measurements with very high errors
# df = df.drop(list(range(20,31)) + `[16, 18, 15, 0])
df

,picture_name,bottom_pixel,bottom_pixel_error,top_pixel,top_pixel_error,DC-kilovolts,DC_error-kilovolts,length-pixels,length_error-pixels,height-pixels,height_error-pixels
0,DSC03175.JPG,1652,15,1576,15,0.656910,0.0001,0,21.213203,38.0,21.213203
1,DSC03176.JPG,1635,15,1518,15,0.715370,0.0001,41,21.213203,75.5,21.213203
2,DSC03177.JPG,1631,15,1466,15,0.755690,0.0001,89,21.213203,103.5,21.213203
3,DSC03178.JPG,1628,15,1404,15,0.805540,0.0001,148,21.213203,136.0,21.213203
4,DSC03179.JPG,1622,15,1346,15,0.850040,0.0001,200,21.213203,168.0,21.213203
5,DSC03180.JPG,1617,15,1285,15,0.900110,0.0001,256,21.213203,201.0,21.213203
6,DSC03181.JPG,1611,15,1221,15,0.952680,0.0001,314,21.213203,236.0,21.213203
7,DSC03182.JPG,1610,15,1144,15,1.009950,0.0001,390,21.213203,275.0,21.213203
8,DSC03183.JPG,1604,15,1089,15,1.051200,0.0001,439,21.213203,305.5,21.213203
9,DSC03184.JPG,1597,15,1017,15,1.109040,0.0001,504,21.213203,345.0,21.213203


Plot length

In [99]:
results, chi_square, degrees_freedom, chi_square_reduced, p_value = odr_fit.perform_odr(
    4850 * (df["DC-kilovolts"].values * 1e+3 - 0.06),
    4850 * (df["DC_error-kilovolts"].values * 1e+3 - 0.06),
    df["length-pixels"].values * pixel_to_mm * 1e-3,
    df["length_error-pixels"].values * pixel_to_mm * 1e-3,
)
print(
    f"Slope: {results.beta[0]} ± {results.sd_beta[0]},\n"
    f"Intercept: {results.beta[1]} ± {results.sd_beta[1]}\n"
    f"Chi-square: {chi_square},\n"
    f"Degreem freedom: {degrees_freedom},\n"
    f"Reduced Chi-square: {chi_square_reduced},\n"
    f"P-value: {p_value}"
)

z_max_slope = results.beta[0]
z_max_slope_error = results.sd_beta[0]

odr_fit.plot_fit(
    4850 * (df["DC-kilovolts"].values * 1e+3 - 0.06),
    4850 * (df["DC_error-kilovolts"].values * 1e+3 - 0.06),
    df["length-pixels"].values * pixel_to_mm * 1e-3,
    df["length_error-pixels"].values * pixel_to_mm * 1e-3,
    results,
    "plot_length_fit.png",
)

odr_fit.plot_residuals(
    4850 * (df["DC-kilovolts"].values * 1e+3 - 0.06),
    4850 * (df["DC_error-kilovolts"].values * 1e+3 - 0.06),
    df["length-pixels"].values * pixel_to_mm * 1e-3,
    df["length_error-pixels"].values * pixel_to_mm * 1e-3,
    results,
    "plot_length_residuals.png",
)

Slope: 1.4279078033140572e-10 ± 1.4254952761017137e-12,
Intercept: -0.00046112623242954736 ± 6.5314172500088536e-06
Chi-square: 47.08846414376795,
Degreem freedom: 27,
Reduced Chi-square: 1.744017190509924,
P-value: 0.009687834593069655


Plot height

In [100]:
results, chi_square, degrees_freedom, chi_square_reduced, p_value = odr_fit.perform_odr(
    4850 * (df["DC-kilovolts"].values * 1e+3 - 0.06),
    4850 * (df["DC_error-kilovolts"].values * 1e+3 - 0.06),
    df["height-pixels"].values * pixel_to_mm * 1e-3,
    df["height_error-pixels"].values * pixel_to_mm * 1e-3,
)
print(
    f"Slope: {results.beta[0]} ± {results.sd_beta[0]},\n"
    f"Intercept: {results.beta[1]} ± {results.sd_beta[1]}\n"
    f"Chi-square: {chi_square},\n"
    f"Degreem freedom: {degrees_freedom},\n"
    f"Reduced Chi-square: {chi_square_reduced},\n"
    f"P-value: {p_value}"
)

z_eq_slope = results.beta[0]
z_eq_slope_error = results.sd_beta[0]

odr_fit.plot_fit(
    4850 * (df["DC-kilovolts"].values * 1e+3 - 0.06),
    4850 * (df["DC_error-kilovolts"].values * 1e+3 - 0.06),
    df["height-pixels"].values * pixel_to_mm * 1e-3,
    df["height_error-pixels"].values * pixel_to_mm * 1e-3,
    results,
    "plot_height_fit.png",
)

odr_fit.plot_residuals(
    4850 * (df["DC-kilovolts"].values * 1e+3 - 0.06),
    4850 * (df["DC_error-kilovolts"].values * 1e+3 - 0.06),
    df["height-pixels"].values * pixel_to_mm * 1e-3,
    df["height_error-pixels"].values * pixel_to_mm * 1e-3,
    results,
    "plot_height_residuals.png",
)

Slope: 8.423234595056637e-11 ± 6.861532076922562e-13,
Intercept: -0.0002487720413103862 ± 3.143856777835233e-06
Chi-square: 10.91004504507526,
Degreem freedom: 27,
Reduced Chi-square: 0.4040757424101948,
P-value: 0.9974264352814214


In [101]:
charge_over_mass2_in_c_over_kg = (z_max_slope ** 2 / z_eq_slope) * (50*50 / 2)
charge_over_mass2_in_c_over_kg

np.float64(3.02573891264076e-07)